### Fact Orders table

**Data Reading**

In [0]:
df = spark.sql("select * from databricks_cata.silver.orders")
display(df)

In [0]:
df_dimCus = spark.sql("select DimCustomerkey , Customer_id as dimcustomer_id from databricks_cata.gold.dimcustomers")

df_dimPro = spark.sql('''select product_id as dimproduct_id , product_id as DimProductkey  
                      from databricks_cata.gold.dimproducts''') 

# display(df_dimPro)

**Fact DataFrame**

In [0]:
df_fact = df.join(df_dimCus, df.customer_id == df_dimCus.dimcustomer_id, 'left').join(df_dimPro, df.product_id == df_dimPro.dimproduct_id, 'left')

df_fact_new = df_fact.drop('dimcustomer_id','dimproduct_id','customer_id','product_id')

In [0]:
df_fact_new.display()

**Upsert fact table**

In [0]:
from delta.tables import *


if spark.catalog.tableExists("databricks_cata.gold.Factorders"):
  
  dlt_obj = DeltaTable.forName(spark, "databricks_cata.gold.Factorders")
  
  dlt_obj.alias("old").merge(df_fact_new.alias("new"), "old.order_id = new.order_id and old.DimCustomerkey = new.DimCustomerkey and old.DimProductkey = new.DimProductkey")\
    .whenMatchedUpdateAll()\
    .whenNotMatchedInsertAll()\
    .execute()

else:
  df_fact_new.write.format("delta")\
    .saveAsTable("databricks_cata.gold.Factorders")



In [0]:
df = spark.sql("select * from databricks_cata.gold.factorders")
display(df)